In [136]:
# carga de librerías
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats  


import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

In [137]:
# función que evalua el modelo
def evaluar_modelo(modelo, X, Y, X_test, Y_test, nombre="Regresión"):
    # Evalúa el modelo
    R2 = modelo.score(X, Y)
    
    # Imprimir las métricas
    print(f"{nombre}:")
    print(f"  - R^2 entrenamiento: {R2}")

    Y_predicho_test = modelo.predict(X_test)
    
    R2_test = r2_score(Y_test, Y_predicho_test)

    print(f"  - R^2 test: {R2_test}")
    print()

In [138]:
# Lectura de datos
df = pd.read_excel("agua.xlsx")
cols = df.columns

# cambiar el nombre de las columnas
df.columns = ["id","p1","p2","p3","p4","p5","p6","p7"]

# Borrar columna p7
df.drop(columns=["p7"], inplace=True)

# Transforma la variable en una variable nueva: si la p6 es mayor de 10 o si es manor de 10
df.loc[df.p6 <= 9, "p6"] = 0
df.loc[df.p6 == 10, "p6"] = 1

# Imputa los datos en blanco de las variables P3 y P4 con la media de cada una de las variables
df.loc[df.p3.isnull(), "p3"] = df.p3.mean()
df.loc[df.p4.isnull(), "p4"] = df.p4.mean()

In [139]:
# conjunto de train
train = df[["p1","p2","p3","p4","p6","p5"]]

# si p5 es null, se pone en otro df y tenemos test
df_preditct_p5_null = df[df["p5"].isnull()]
test = df_preditct_p5_null

train.dropna(how = 'any', inplace=True)
train.isna().sum()

# definimos y_tran, y_test, x_train, x_test, también quitamos la columna de id
y_train = train["p5"]
x_train = train.drop(columns=["p5"])
x_test = test.drop(columns=["p5","id"])
y_test = test["p5"]

# comprobamos los tamaños de los df 
print("x_train.shape", x_train.shape)
print("y_train.shape", y_train.shape)
print("x_test.shape", x_test.shape)
print("y_test.shape", y_test.shape)

x_train.shape (27, 5)
y_train.shape (27,)
x_test.shape (2, 5)
y_test.shape (2,)


/var/folders/hk/90bb_3190r3gc50_fdt1k3340000gn/T/ipykernel_44101/1714503016.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.dropna(how = 'any', inplace=True)


In [140]:
###############################################################
# Para imputar los datos en blanco de la variable P5, se utiliza un metodo de regresión como primer modelo
###############################################################
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)

# Evaluación del modelo
Y_pred_entrenamiento_m1 = model.predict(x_train)
Y_pred_test_m1 = model.predict(x_test)

# Evaluación del modelo con score
R2 = model.score(x_train, y_train)
print("R2", R2)

# MSE
mse = mean_squared_error(y_train, Y_pred_entrenamiento_m1)
print("mse", mse)

###############################################################
# hacer otro método de regresión que no sea el anterior, cogemos Rigde por ejemplo (podriamos utilizar otro)
###############################################################

from sklearn.linear_model import Ridge

model = Ridge()
model.fit(x_train, y_train)

# Evaluación del modelo
Y_pred_entrenamiento_m2 = model.predict(x_train)
Y_pred_test_m2 = model.predict(x_test)

# Evaluación del modelo con score
R2 = model.score(x_train, y_train)
print("R2", R2)

# MSE
mse = mean_squared_error(y_train, Y_pred_entrenamiento_m2)
print("mse", mse)

# Como ambos modelos son muy similares en terminos de R2 y MSE, se puede utilizar cualquiera de los dos.
# Cogemos el primer modelo
# Se imputan las predicciones en blanco de la variable P5 utilizando la predicción del modelo 1
df.loc[df.p5.isnull(), "p5"] = Y_pred_test_m1

R2 0.8542983436685863
mse 0.3745471933677218
R2 0.8536962148330891
mse 0.3760950526787255


In [149]:
# Ejercicio 4

# Los primeros 30 registros son el conjunto de train y el resto el conjunto de test
train = df.head(29)
test = df.tail(10)

# definimos y_tran, y_test, x_train, x_test
y_train = train["p6"]
x_train = train.drop(columns=["p6","id"])
x_test = test.drop(columns=["p6","id"])
y_test = test["p6"]

print("x_train.shape", x_train.shape)
print("y_train.shape", y_train.shape)
print("x_test.shape", x_test.shape)
print("y_test.shape", y_test.shape)


x_train.shape (29, 5)
y_train.shape (29,)
x_test.shape (10, 5)
y_test.shape (10,)


In [152]:
###############################################################
# Hacer un modelo de decision tree para predecir la variable P6
###############################################################
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x_train, y_train)

# Evaluación del modelo
Y_pred_entrenamiento_m1 = model.predict(x_train)
Y_pred_test_m1 = model.predict(x_test)

# sacar la matriz de confusión con Y_pred_entrenamiento_m1 y y_train
from sklearn.metrics import confusion_matrix
mc=confusion_matrix(y_train, Y_pred_entrenamiento_m1)
print(mc)

# obtener accuracy
accuracy = accuracy_score(y_train, Y_pred_entrenamiento_m1)
print("accuracy modelo1", accuracy)

###############################################################
# Hacer otro modelo para la variable P6 con regresión logistica
###############################################################

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)

# Evaluación del modelo
Y_pred_entrenamiento_m2 = model.predict(x_train)
Y_pred_test_m2 = model.predict(x_test)

# Matriz de confusión
mc=confusion_matrix(y_train, Y_pred_entrenamiento_m2)
print(mc)

# obtener accuracy para evaluar el modelo
accuracy = accuracy_score(y_train, Y_pred_entrenamiento_m2)
print("accuracy modelo2", accuracy)

# Como ambos modelos son muy similares en terminos de accuracy, se puede utilizar cualquiera de los dos.
# Cogemos el primer modelo de DecisionTreeClassifier
# Se imputan las predicciones en blanco de la variable P6 utilizando la predicción del modelo 1
df.loc[df.p6.isnull(), "p6"] = Y_pred_test_m1

[[18  0]
 [ 1 10]]
accuracy modelo1 0.9655172413793104
[[17  1]
 [ 0 11]]
accuracy modelo2 0.9655172413793104


In [154]:
# Vemos como la variable p6 ha sido imputada correctamente los valores según los datos predichos
df.tail(10)

,id,p1,p2,p3,p4,p5,p6
29,30,10,9,10.0,10.0,10.0,0.0
30,31,10,10,10.0,10.0,10.0,1.0
31,32,10,10,10.0,10.0,10.0,1.0
32,33,9,9,7.0,9.0,7.0,0.0
33,34,8,8,10.0,10.0,9.0,0.0
34,35,8,10,10.0,10.0,8.0,0.0
35,36,10,10,10.0,10.0,10.0,1.0
36,37,10,10,4.0,10.0,10.0,1.0
37,38,5,3,9.0,8.0,5.0,0.0
38,39,10,10,10.0,10.0,10.0,1.0
